In [1]:
pip install rdkit-pypi

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\82106\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import pymysql
import pandas as pd

# MySQL 서버 연결 정보
connection = pymysql.connect(
    host='127.0.0.1',    
    port=3307,           
    user='root',         
    password='123',  
    database='my_database'   
)

In [3]:
query = "SELECT * FROM train;"  # 'train' 테이블에서 데이터를 가져오는 쿼리
train_df = pd.read_sql(query, connection)

C:\Users\82106\AppData\Local\Temp\ipykernel_46772\2256127756.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  train_df = pd.read_sql(query, connection)


In [4]:
from rdkit import Chem
from rdkit.Chem import Descriptors

In [5]:
# 분자 특성 계산 함수 정의
def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None, None, None
    
    # 분자량 계산 
    molecular_weight = Descriptors.MolWt(mol) #mol= smiles의 분자 객체
    # LogP 계산 (분자의 소수성 지표)
    logp = Descriptors.MolLogP(mol)
    # 수소 결합 수용체 수
    num_h_acceptors = Descriptors.NumHAcceptors(mol)
    # 수소 결합 공여체 수
    num_h_donors = Descriptors.NumHDonors(mol)
    
    return molecular_weight, logp, num_h_acceptors, num_h_donors

**각 SMILES 문자열에 대해 MolecularWeight, LogP, NumHAcceptors, NumHDonors 값을 계산하여 데이터프레임에 새로운 열로 추가**

In [9]:
# 각 SMILES에 대해 특성 계산
train_df['MolecularWeight'], train_df['LogP'], train_df['NumHAcceptors'], train_df['NumHDonors'] = zip(
    *train_df['Smiles'].apply(calculate_descriptors)
)

In [12]:
train_df.isnull().sum()

癤풫olecule ChEMBL ID    0
Standard Type          0
Standard Relation      0
Standard Value         0
Standard Units         0
pChEMBL Value          0
Assay ChEMBL ID        0
Target ChEMBL ID       0
Target Name            0
Target Organism        0
Target Type            0
Document ChEMBL ID     0
IC50_nM                0
pIC50                  0
Smiles                 0
MolecularWeight        0
LogP                   0
NumHAcceptors          0
NumHDonors             0
dtype: int64

결측치 없음

In [13]:
# 새롭게 생성된 데이터 CSV 파일로 저장
train_df.to_csv('C:/Users/82106/Desktop/데이콘/제 2회 신약개발/데이터/open/molecular_descriptors.csv', index=False)